# Webscrapping 1

In [ ]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
from datetime import datetime
import pandas as pd

In [2]:
#creating a useragent
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}

In [3]:
#H&M site catalog
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
# id # product_name # product_type # product_color # price
page = requests.get(url, headers = header)
print(page)

<Response [200]>


In [4]:
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
products = soup.find('ul', class_= 'products-listing small')

In [90]:
product_list = products.find_all('article', class_ = 'hm-product-item')

In [62]:
product_id = [p.get('data-articlecode') for p in product_list]

In [63]:
product_category = [p.get('data-category') for p in product_list]

In [64]:
product_list = products.find_all('a', class_ = 'link')
product_name = [p.get_text() for p in product_list]

In [70]:
product_list = products.find_all('span', class_ = 'price regular')
product_price = [p.get_text() for p in product_list]

In [ ]:
#product color

In [ ]:
#product composition

In [85]:
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T  #transposição de coluna para linha
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']
data['scrapy_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [89]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_time
0,0985197001,men_jeans_slim,Slim Jeans,$ 19.99,2021-12-07 11:58:51
1,0985159002,men_jeans_skinny,Skinny Jeans,$ 19.99,2021-12-07 11:58:51
2,1028865001,men_jeans_relaxed,Relaxed Jeans with Embroidery Detail,$ 49.99,2021-12-07 11:58:51
3,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-12-07 11:58:51
4,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-12-07 11:58:51


# Webscrapping 2

In [ ]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
from datetime import datetime
import pandas as pd
import numpy as np

In [ ]:
#creating a useragent
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
page = requests.get(url, headers = header)
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
total_items = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')

In [ ]:
int(total_items)

In [ ]:
#how many pages to get all data?
page_number = np.round(int(total_items)/36)

In [ ]:
url2 = url + '?page-size' + str(int(page_number) *36)
url2

# Webscrapping 3

In [26]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
from datetime import datetime
import pandas as pd
import numpy as np

In [27]:
#one single product test

In [28]:
#creating a useragent
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}
#API Request
url = 'https://www2.hm.com/en_us/productpage.0985197001.html'
page = requests.get(url, headers = header)
#BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

In [29]:
########################   color name   ####################################
#including first prod (class filter-option miniature active)

product_list = soup.find_all('a', class_ = 'filter-option miniature active') + soup.find_all('a', class_ = 'filter-option miniature')
color_item = [p.get('data-color') for p in product_list]

#product id
product_id = [p.get('data-articlecode') for p in product_list]

#creating data frame with product id+color name
df_color = pd.DataFrame((product_id, color_item)).T
df_color.columns = ['product_id', 'color_name'] #renaming columns

#generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

#####################     composition     #################################3
product_composition_list = soup.find_all('div', class_ = 'pdp-description-list-item')

product_composition = [list(filter(None, p.get_text().split("\n"))) for p in product_composition_list]

df_composition = pd.DataFrame(product_composition).T

#renaming labels
df_composition.columns = df_composition.iloc[0]

#deleting first row
df_composition = df_composition.iloc[1:].fillna(method = 'ffill')

#generate style id +color id

df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

#merge df_color and df_composition
df_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how = 'left', on = 'style_id')

In [30]:
df_sku

,product_id,color_name,style_id,color_id,Fit,Composition
0,0985197001,Black,0985197,001,Slim fit,Pocket lining: Cotton 100%
1,0985197001,Black,0985197,001,Slim fit,"Shell: Cotton 98%, Spandex 2%"
2,0985197002,Midnight blue,0985197,002,Slim fit,Pocket lining: Cotton 100%
3,0985197002,Midnight blue,0985197,002,Slim fit,"Shell: Cotton 98%, Spandex 2%"
4,0985197003,Denim blue,0985197,003,Slim fit,Pocket lining: Cotton 100%
5,0985197003,Denim blue,0985197,003,Slim fit,"Shell: Cotton 98%, Spandex 2%"
6,0985197004,Dark denim blue,0985197,004,Slim fit,Pocket lining: Cotton 100%
7,0985197004,Dark denim blue,0985197,004,Slim fit,"Shell: Cotton 98%, Spandex 2%"
8,0985197005,Dark denim blue,0985197,005,Slim fit,Pocket lining: Cotton 100%
9,0985197005,Dark denim blue,0985197,005,Slim fit,"Shell: Cotton 98%, Spandex 2%"
